<a href="https://colab.research.google.com/github/LeoDinga/DL_Project/blob/main/Project_DL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !git clone https://github.com/THETIS-dataset/dataset.git
# !mv dataset data


!git clone --filter=blob:none --no-checkout https://github.com/THETIS-dataset/dataset.git
%cd dataset
!git sparse-checkout init --cone
!git sparse-checkout set VIDEO_Skelet3D
!git checkout

Cloning into 'dataset'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 113 (delta 0), reused 2 (delta 0), pack-reused 110 (from 1)
Receiving objects: 100% (113/113), 214.90 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/dataset/dataset
remote: Enumerating objects: 1217, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1217 (delta 0), reused 0 (delta 0), pack-reused 1216 (from 1)
Receiving objects: 100% (1217/1217), 507.08 MiB | 20.81 MiB/s, done.
Updating files: 100% (1218/1218), done.
Your branch is up to date with 'origin/main'.


In [22]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is inco

In [3]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import shutil
import random
#import mediapipe as mp

In [11]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: False
Device name: No GPU


In [8]:
import os
print(os.listdir('VIDEO_Skelet3D'))

['smash', 'flat_service', 'forehand_volley', 'backhand', 'backhand_volley', 'forehand_flat', 'slice_service', 'backhand_slice', 'forehand_openstands', 'kick_service', 'backhand2hands', 'forehand_slice']


In [10]:
#shows the videos in each folder
data_path = 'VIDEO_Skelet3D'

# Print out the directory structure
for root, dirs, files in os.walk(data_path):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    print("-" * 40)

Root: VIDEO_Skelet3D
Dirs: ['smash', 'flat_service', 'forehand_volley', 'backhand', 'backhand_volley', 'forehand_flat', 'slice_service', 'backhand_slice', 'forehand_openstands', 'kick_service', 'backhand2hands', 'forehand_slice']
Files: []
----------------------------------------
Root: VIDEO_Skelet3D/smash
Dirs: []
Files: ['p16_smash_skelet3D_s1.avi', 'p23_smash_skelet3D_s2.avi', 'p31_smash_skelet3D_s2.avi', 'p38_smash_skelet3D_s3.avi', 'p53_smash_skelet3D_s3.avi', 'p30_smash_skelet3D_s3.avi', 'p38_smash_skelet3D_s2.avi', 'p24_smash_skelet3D_s2.avi', 'p47_smash_skelet3D_s3.avi', 'p51_smash_skelet3D_s2.avi', 'p26_smash_skelet3D_s2.avi', 'p6_smash_skelet3D_s3.avi', 'p17_smash_skelet3D_s2.avi', 'p39_smash_skelet3D_s2.avi', 'p52_smash_skelet3D_s2.avi', 'p43_smash_skelet3D_s3.avi', 'p34_smash_skelet3D_s3.avi', 'p36_smash_skelet3D_s3.avi', 'p44_smash_skelet3D_s2.avi', 'p2_smash_skelet3D_s2.avi', 'p7_smash_skelet3D_s3.avi', 'p1_smash_skelet3D_s3.avi', 'p12_smash_skelet3D_s2.avi', 'p24_smash_s

In [12]:
def convert_video_to_npy(video_path, resize_shape=(224, 224)):
    """
    Converts a video to a numpy array of resized frames.

    Parameters:
    - video_path: The path to the video file.
    - resize_shape: The desired frame size (default is 224x224).

    Returns:
    - frames_array: Numpy array containing all frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to the desired shape
        frame_resized = cv2.resize(frame, resize_shape)
        frames.append(frame_resized)

    cap.release()

    # Convert list of frames into a numpy array
    frames_array = np.array(frames)
    return frames_array

def create_subfolders(pastas_dir, output_dir):
    """
    Creates subfolders in the output directory and processes .avi files.

    Parameters:
    - pastas_dir: The root directory containing class folders.
    - output_dir: The directory where the .npy files will be saved.
    """
    for pastas_filename in os.listdir(pastas_dir):
        origem_path = os.path.join(pastas_dir, pastas_filename)

        # Check if it's a directory (ignore files)
        if os.path.isdir(origem_path):
            nova_pasta_path = os.path.join(output_dir, pastas_filename)
            os.makedirs(nova_pasta_path, exist_ok=True)

            # Process each video file in the subfolder
            videos_dir = os.path.join(pastas_dir, pastas_filename)
            for video_filename in os.listdir(videos_dir):
                if video_filename.endswith(".avi"):  # Process only .avi files
                    video_path = os.path.join(videos_dir, video_filename)
                    video_name = os.path.splitext(video_filename)[0]

                    try:
                        # Convert the video to numpy array
                        frames_array = convert_video_to_npy(video_path)

                        # Save the numpy array to a .npy file
                        output_filename = os.path.join(nova_pasta_path, f"{video_name}.npy")
                        np.save(output_filename, frames_array)
                    except ValueError as e:
                        print(f"Error processing {video_filename}: {e}")
                    except Exception as e:
                        print(f"Unexpected error with {video_filename}: {e}")

# Example usage
pastas_dir = 'VIDEO_Skelet3D'  # Directory with .avi videos
npy_dir = 'npy_videos'  # Directory to save .npy files
os.makedirs(npy_dir, exist_ok=True)

# Process videos and maintain class folders
create_subfolders(pastas_dir, npy_dir)

In [24]:
import mediapipe as mp
from tqdm.auto import tqdm
# Define the root directory containing the action folders
root_dir = "npy_videos"

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

all_keypoints = {}  # Dictionary to store keypoints for each video

# Iterate through the subdirectories (action categories)
for action_folder in tqdm(os.listdir(root_dir), desc="Processing actions"):
    action_path = os.path.join(root_dir, action_folder)
    if os.path.isdir(action_path):
        all_keypoints[action_folder] = {}
        # Iterate through the .npy files (videos) within each action folder
        for video_file in tqdm(os.listdir(action_path), desc=f"Processing videos in {action_folder}", leave=False):
            if video_file.endswith(".npy"):
                video_path = os.path.join(action_path, video_file)
                try:
                    sample = np.load(video_path)  # Shape: (num_frames, height, width, channels)
                    video_keypoints = []

                    for frame in sample:
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        results = pose.process(frame_rgb)

                        if results.pose_landmarks:
                            frame_keypoints = []
                            for lm in results.pose_landmarks.landmark:
                                frame_keypoints.append([lm.x, lm.y, lm.z])
                            video_keypoints.append(frame_keypoints)
                        else:
                            # If detection fails, fill with zeros (33 keypoints × 3 coords)
                            video_keypoints.append(np.zeros((33, 3)))

                    video_keypoints = np.array(video_keypoints)  # Shape: (num_frames, 33, 3)
                    all_keypoints[action_folder][video_file] = video_keypoints
                    # print(f"Keypoints shape for {video_file}: {video_keypoints.shape}")

                except Exception as e:
                    print(f"Error processing {video_file} in {action_folder}: {e}")

# Print the structure of the resulting dictionary
print("\nStructure of all_keypoints:")
for action, videos in all_keypoints.items():
    print(f"- {action}: {len(videos)} videos")
    # You can optionally print the shape of the first video's keypoints in each action
    for video, kps in videos.items():
        print(f"  - {video}: {kps.shape}")
        break # Only print the first video's shape for brevity

Processing actions:   0%|          | 0/12 [00:00<?, ?it/s]

Processing videos in smash:   0%|          | 0/104 [00:00<?, ?it/s]

Processing videos in flat_service:   0%|          | 0/96 [00:00<?, ?it/s]

Processing videos in forehand_volley:   0%|          | 0/93 [00:00<?, ?it/s]

Processing videos in backhand:   0%|          | 0/97 [00:00<?, ?it/s]

...

Processing videos in forehand_openstands:   0%|          | 0/101 [00:00<?, ?it/s]

Processing videos in kick_service:   0%|          | 0/109 [00:00<?, ?it/s]

Processing videos in backhand2hands:   0%|          | 0/107 [00:00<?, ?it/s]

Processing videos in forehand_slice:   0%|          | 0/97 [00:00<?, ?it/s]


Structure of all_keypoints:
- smash: 104 videos
  - p50_smash_skelet3D_s2.npy: (78, 33, 3)
- flat_service: 96 videos
  - p11_serflat_skelet3D_s3.npy: (64, 33, 3)
- forehand_volley: 93 videos
  - p24_fvolley_skelet3D_s2.npy: (84, 33, 3)
- backhand: 97 videos
  - p17_backhand_skelet3D_s3.npy: (37, 33, 3)
- backhand_volley: 103 videos
  - p14_bvolley_skelet3D_s3.npy: (80, 33, 3)
- forehand_flat: 110 videos
  - p10_foreflat_skelet3D_s3.npy: (95, 33, 3)
- slice_service: 100 videos
  - p29_serslice_skelet3D_s3.npy: (55, 33, 3)
- backhand_slice: 100 videos
  - p46_bslice_skelet3D_s3.npy: (68, 33, 3)
- forehand_openstands: 101 videos
  - p47_foreopen_skelet3D_s3.npy: (56, 33, 3)
- kick_service: 109 videos
  - p17_serkick_skelet3D_s2.npy: (55, 33, 3)
- backhand2hands: 107 videos
  - p37_backhand2h_skelet3D_s3.npy: (65, 33, 3)
- forehand_slice: 97 videos
  - p31_fslice_skelet3D_s2.npy: (74, 33, 3)


# not sure about the things below!!!

In [ ]:
# Saving Results to Separate NumPy Files:

output_root_dir = "processed_keypoints"
os.makedirs(output_root_dir, exist_ok=True)

for action, videos in all_keypoints.items():
    action_output_dir = os.path.join(output_root_dir, action)
    os.makedirs(action_output_dir, exist_ok=True)
    for video_file, keypoints in videos.items():
        output_path = os.path.join(action_output_dir, video_file)
        np.save(output_path, keypoints)
        # print(f"Saved keypoints for {video_file} in {action} to {output_path}")

print(f"All keypoint data saved to the '{output_root_dir}' directory.")

In [23]:
import mediapipe as mp

# Load video frames
sample = np.load("npy_videos/backhand2hands/p10_backhand2h_skelet3D_s2.npy")  # shape (72, 224, 224, 3)

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

keypoints = []

for frame in sample:
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        frame_keypoints = []
        for lm in results.pose_landmarks.landmark:
            frame_keypoints.append([lm.x, lm.y, lm.z])
        keypoints.append(frame_keypoints)
    else:
        # If detection fails, fill with zeros (33 keypoints × 3 coords)
        keypoints.append(np.zeros((33, 3)))

keypoints = np.array(keypoints)  # Shape: (72, 33, 3)

print("Keypoints shape:", keypoints.shape)

Keypoints shape: (72, 33, 3)


In [14]:
def extract_frames_from_npy(npy_file, num_frames=120):
    """
    Extracts evenly spaced frames from a numpy array (video frames).

    Parameters:
    - npy_file: Path to the .npy file containing video frames.
    - num_frames: Number of frames to extract (default is 120).

    Returns:
    - extracted_frames: The selected frames as a numpy array.
    """
    # Load the numpy array (which contains all frames of the video)
    frames = np.load(npy_file)

    # Calculate indices to extract 120 evenly spaced frames
    total_frames = frames.shape[0]
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    # Extract the frames
    extracted_frames = frames[frame_indices]

    return extracted_frames

def process_npy_videos_to_frames(npy_dir, output_dir, num_frames=120):
    """
    Processes .npy files in the directory, extracting a specified number of frames from each video.

    Parameters:
    - npy_dir: The directory containing the class subfolders with .npy files.
    - output_dir: The directory to save the extracted frames (with the class subfolder structure).
    - num_frames: Number of frames to extract (default is 120).
    """
    # Loop through each class folder in the npy_dir
    for npy_subfolder in os.listdir(npy_dir):
        subfolder_path = os.path.join(npy_dir, npy_subfolder)

        # Ensure it's a directory (class folder)
        if os.path.isdir(subfolder_path):
            # Create the corresponding output class folder in the output_dir
            output_subfolder_path = os.path.join(output_dir, npy_subfolder)
            os.makedirs(output_subfolder_path, exist_ok=True)

            # Loop through each .npy file in the subfolder
            for npy_filename in os.listdir(subfolder_path):
                if npy_filename.endswith(".npy"):  # Process only .npy files
                    npy_file = os.path.join(subfolder_path, npy_filename)

                    # Extract the frames from the .npy file
                    extracted_frames = extract_frames_from_npy(npy_file, num_frames)

                    # Save the extracted frames as a new .npy file in the corresponding subfolder
                    output_filename = os.path.join(output_subfolder_path, npy_filename)
                    np.save(output_filename, extracted_frames)
                    print(f"Saved 120 frames from {npy_filename} as {output_filename}")

# Example usage:
npy_dir = 'npy_videos'  # Directory with .npy files (containing class subfolders)
output_dir = '120_frames'  # Directory to save 120 frames, maintaining the class structure
os.makedirs(output_dir, exist_ok=True)

# Process videos and extract frames
process_npy_videos_to_frames(npy_dir, output_dir)

Saved 120 frames from p50_smash_skelet3D_s2.npy as 120_frames/smash/p50_smash_skelet3D_s2.npy
Saved 120 frames from p25_smash_skelet3D_s3.npy as 120_frames/smash/p25_smash_skelet3D_s3.npy
Saved 120 frames from p55_smash_skelet3D_s1.npy as 120_frames/smash/p55_smash_skelet3D_s1.npy
Saved 120 frames from p8_smash_skelet3D_s3.npy as 120_frames/smash/p8_smash_skelet3D_s3.npy
Saved 120 frames from p35_smash_skelet3D_s3.npy as 120_frames/smash/p35_smash_skelet3D_s3.npy
Saved 120 frames from p15_smash_skelet3D_s3.npy as 120_frames/smash/p15_smash_skelet3D_s3.npy
Saved 120 frames from p42_smash_skelet3D_s1.npy as 120_frames/smash/p42_smash_skelet3D_s1.npy
Saved 120 frames from p32_smash_skelet3D_s3.npy as 120_frames/smash/p32_smash_skelet3D_s3.npy
Saved 120 frames from p20_smash_skelet3D_s3.npy as 120_frames/smash/p20_smash_skelet3D_s3.npy
Saved 120 frames from p12_smash_skelet3D_s2.npy as 120_frames/smash/p12_smash_skelet3D_s2.npy
Saved 120 frames from p27_smash_skelet3D_s3.npy as 120_frames/

In [15]:
def split_by_player(video_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    # Step 1: Get a list of all players
    players = set()  # Set to store unique players (assuming folder names represent players)

    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):  # Only process .npy files
            player_name = video_filename.split('_')[0]  # Assuming player info is in the filename
            players.add(player_name)

    # Step 2: Shuffle the list of players
    players = list(players)
    random.shuffle(players)

    # Step 3: Split players into train/val/test sets
    num_players = len(players)
    train_size = int(train_ratio * num_players)
    val_size = int(val_ratio * num_players)

    train_players = players[:train_size]
    val_players = players[train_size:train_size + val_size]
    test_players = players[train_size + val_size:]

    # Step 4: Create directories for train/val/test sets
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Step 5: Move videos to their respective directories
    for video_filename in os.listdir(video_dir):
        if video_filename.endswith(".npy"):
            player_name = video_filename.split('_')[0]  # Get the player name
            src_path = os.path.join(video_dir, video_filename)

            if player_name in train_players:
                dst_dir = train_dir
            elif player_name in val_players:
                dst_dir = val_dir
            else:
                dst_dir = test_dir

            # Move the file to the appropriate directory
            shutil.copy(src_path, os.path.join(dst_dir, video_filename))
            #(f"Moved {video_filename} to {dst_dir}")

# Example usage:
video_dir = '120_frames'  # Directory containing all the .npy files
output_dir = 'splits'  # Directory where train/val/test splits will be saved
os.makedirs(output_dir, exist_ok=True)

split_by_player(video_dir, output_dir)

In [ ]:
!rm -r data/VIDEO_RGB data/VIDEO_Depth data/VIDEO_Mask data/VIDEO_Skelet2D data/VIDEO_Skelet3D

In [20]:
sample = np.load("npy_videos/backhand2hands/p10_backhand2h_skelet3D_s2.npy")
print(sample.shape)

(72, 224, 224, 3)


In [16]:
import numpy as np
sample = np.load("120_frames/backhand2hands/p10_backhand2h_skelet3D_s2.npy")
print(sample.shape)

(120, 224, 224, 3)


In [14]:
import mediapipe as mp
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def extract_pose_from_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        joints = []
        for lm in results.pose_landmarks.landmark:
            joints.append([lm.x, lm.y, lm.z])
        return np.array(joints)
    else:
        return None

def process_video_to_skeleton(video_path, num_frames=120):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        joints = extract_pose_from_frame(frame)
        if joints is not None:
            frames.append(joints)

    cap.release()

    # Interpolate to 120 frames
    if len(frames) < num_frames:
        # Pad with last frame
        while len(frames) < num_frames:
            frames.append(frames[-1])
    else:
        idxs = np.linspace(0, len(frames) - 1, num_frames, dtype=int)
        frames = [frames[i] for i in idxs]

    return np.array(frames)  # shape: (120, 33, 3)

# Save skeletons
input_folder = 'data/VIDEO_Skelet3D'
output_folder = 'data/skeletons'
os.makedirs(output_folder, exist_ok=True)

for folder in os.listdir(input_folder):
    subfolder = os.path.join(input_folder, folder)
    out_subfolder = os.path.join(output_folder, folder)
    os.makedirs(out_subfolder, exist_ok=True)

    for file in os.listdir(subfolder):
        if file.endswith('.avi'):
            video_path = os.path.join(subfolder, file)
            skel = process_video_to_skeleton(video_path)
            np.save(os.path.join(out_subfolder, file.replace('.avi', '.npy')), skel)
            print(f"Saved: {file}")

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
!git clone https://github.com/yysijie/st-gcn.git
!cd st-gcn
!pip install -r /content/st-gcn/requirements.txt

In [ ]:
%cd /content/st-gcn

In [ ]:
ls /content/st-gcn/models/pose/coco

In [ ]:
!grep -r "class Model" /content/st-gcn/


In [ ]:
!python main.py recognition -c config/st_gcn/kinetics-skeleton/test.yaml

In [ ]:
!python main.py --config config/stgcn_kinetics.yaml

In [ ]:
!pip install scipy scikit-learn

In [ ]:
!pip install stgcn

In [ ]:
import torch
from stgcn.net.st_gcn import Model  # Assuming stgcn.py is available from the implementation
from torch import nn

# Load a pre-trained ST-GCN model (Kinetics-Pretrained)
def load_stgcn_model(pretrained=True):
    # Assuming you're using a pre-trained model from Kinetics
    model = Model(num_class=60, num_point=18, in_channels=3, graph_size=18)  # Adjust as per your dataset
    if pretrained:
        model.load_state_dict(torch.load('path_to_pretrained_kinetics_model.pth'))
    model.eval()  # Set the model to evaluation mode
    return model

# Example usage:
stgcn_model = load_stgcn_model(pretrained=True)

In [ ]:

class KeypointDataset(Dataset):
    def __init__(self, npy_dir, split='train'):
        self.npy_dir = npy_dir
        self.split = split
        self.files = [f for f in os.listdir(npy_dir) if f.endswith('.npy')]

        # If you have a specific player split, apply that here (e.g., train/val/test)
        if self.split == 'train':
            # Load the train files
            self.files = [f for f in self.files if 'player1' in f or 'player2' in f]  # Example player-based split
        elif self.split == 'val':
            # Load the validation files
            self.files = [f for f in self.files if 'player3' in f or 'player4' in f]  # Example player-based split
        else:
            # Load the test files
            self.files = [f for f in self.files if 'player5' in f or 'player6' in f]  # Example player-based split

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        video_filename = self.files[idx]
        video_path = os.path.join(self.npy_dir, video_filename)

        # Load the video keypoints data
        frames = np.load(video_path)  # Shape (T, N, 3) => (120, num_keypoints, 3)

        # Normalize or preprocess data if needed
        frames = torch.tensor(frames, dtype=torch.float32)

        # Assume labels are stored in a dictionary or follow a certain convention
        label = get_label_from_filename(video_filename)  # You should define this function based on your dataset

        return frames, label

# Define the dataset and dataloaders
train_dataset = KeypointDataset(npy_dir='data/splits/train', split='train')
val_dataset = KeypointDataset(npy_dir='data/splits/val', split='val')
test_dataset = KeypointDataset(npy_dir='data/splits/test', split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3):
    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()  # Assuming classification task

    best_val_acc = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_acc = 0
        train_samples = 0

        # Training loop
        for data, label in train_loader:
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Compute loss and backpropagate
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(output, 1)
            train_acc += (predicted == label).sum().item()
            train_samples += label.size(0)

            train_loss += loss.item()

        # Calculate training accuracy
        train_acc /= train_samples
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        val_acc = validate_model(model, val_loader)
        print(f"Validation Accuracy: {val_acc:.4f}")

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_stgcn_model.pth')

def validate_model(model, val_loader):
    model.eval()
    val_acc = 0
    val_samples = 0

    with torch.no_grad():
        for data, label in val_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            val_acc += (predicted == label).sum().item()
            val_samples += label.size(0)

    val_acc /= val_samples
    return val_acc



### BEFORE TRAINING.


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(DEVICE)


# Start training
train_model(stgcn_model, train_loader, val_loader)


In [ ]:
def test_model(model, test_loader):
    model.eval()
    test_acc = 0
    test_samples = 0

    with torch.no_grad():
        for data, label in test_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            test_acc += (predicted == label).sum().item()
            test_samples += label.size(0)

    test_acc /= test_samples
    print(f"Test Accuracy: {test_acc:.4f}")

# Test the model
test_model(stgcn_model, test_loader)
